<a href="https://colab.research.google.com/github/AlcirCastro/Trabalho-Final-IA/blob/main/TrabalhoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install LTNtorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.9 MB/s eta 0:00:00


In [318]:
class BoundingBoxDataset(torch.utils.data.Dataset):
    def __init__(self, num_samples, image_size=(100, 100), bbox_size=(10, 10)):
        self.num_samples = num_samples
        self.image_size = image_size
        self.bbox_size = bbox_size

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Gerar uma imagem aleatória
        image = torch.rand(*self.image_size)

        # Gerar bounding box aleatório
        bbox_x = round(torch.rand(1).item(), 2)
        bbox_y = round(torch.rand(1).item(), 2)
        bbox_w = round(torch.rand(1).item() * (self.image_size[1] - self.bbox_size[1]) / self.image_size[1], 2)
        bbox_h = round(torch.rand(1).item() * (self.image_size[0] - self.bbox_size[0]) / self.image_size[0], 2)
        bbox = torch.tensor([bbox_x, bbox_y, bbox_w, bbox_h]).float()

        return image, bbox

In [338]:
import torch
import torch.nn as nn
import torch.optim as optim

# Defina os predicados espaciais como funções regulares
# Predicados espaciais
def O(bb1, bb2):
    # Predicate to check if two bounding boxes overlap
    p = ltn.Predicate(func=lambda A, B: torch.all(torch.stack([
        torch.gt(A[:, 0] + A[:, 2] - B[:, 0], 0),  # Verifica se lado direito de A está à direita de B
        torch.lt(A[:, 0] - (B[:, 0] + B[:, 2]), 0),  # Verifica se lado esquerdo de A está à esquerda de B
        torch.gt(A[:, 1] + A[:, 3] - B[:, 1], 0),  # Verifica se lado superior de A está acima de B
        torch.lt(A[:, 1] - (B[:, 1] + B[:, 3]), 0)  # Verifica se lado inferior de A está abaixo de B
    ])))
    condition = p(bb1, bb2)
    return condition

def verificaAbove(bb1, bb2):
  condition1 = ltn.Predicate(func=lambda A, B: torch.all(torch.stack([
        torch.gt(A[:, 1], B[:, 1] + B[:, 3])   # Verifica se lado superior de A está abaixo de B
  ])))

  return condition1(bb1,bb2)


left = ltn.Predicate(func=lambda A, D: torch.lt(A[:, 0] + A[:, 2], D[:, 0]))

def inside(A,C):
    return PO(A,C)

def above(A, C):
    # Predicate to check if A is above C
    return And(Not(O(A, C)), verificaAbove(A,C))

def OutSideLeftAbove(A,C):
    return And(And(above(A,C), left(A,C)), Not(O(A,C)))

def OutSideRightAbove(A,C):
    return And(And(above(A,C), Not(left(A,C))), Not(O(A,C)))

def inSideRight(E, A):
    # Predicado para verificar se E está à direita de A
    return And(O(E, A), Not(left(E, A)))

def inSideLeft(I, A):
    # Predicado para verificar se I está à esquerda de A
    return And(O(I, A), left(I, A))

def OutSideLeft(I, A):
    # Predicado para verificar se I está à esquerda de A
    return And(Not(O(I, A)), left(I, A))

def OutSideRight(I, A):
    # Predicado para verificar se I está à esquerda de A
    return And(Not(O(I, A)), left(A, I))


def interpret_output(output):
    # Se o valor do resultado for maior que 0.5, retorna True, caso contrário, retorna False
    return output.value > 0.5



class PredicadoAprendido(nn.Module):
    def __init__(self):
        super(PredicadoAprendido, self).__init__()
        self.fc1 = nn.Linear(9, 5)  # A entrada agora tem 3 dimensões (O, verificaAbove, above)
        self.fc2 = nn.Linear(5, 4)

    def forward(self, A, B):
        """A: bounding box 1, B: bounding box 2"""
        A = ltn.Variable("A", A)
        B = ltn.Variable("B", B)
        # Converte os objetos LTNObject para tensores PyTorch
        O_tensor = O(A, B).value
        verificaAbove_tensor = verificaAbove(A, B).value
        above_tensor = above(A, B).value
        OutSideLeftAbove_tensor = OutSideLeftAbove(A,B).value
        OutSideRightAbove_tensor = OutSideRightAbove(A,B).value
        inSideRight_tensor = inSideRight(A,B).value
        inSideLeft_tensor = inSideLeft(A,B).value
        OutSideLeft_tensor = OutSideLeft(A,B).value
        OutSideRight_tensor = OutSideRight(A,B).value

        # Concatena os tensores
        features = torch.cat((O_tensor, verificaAbove_tensor, above_tensor,OutSideRight_tensor,OutSideLeft_tensor,inSideLeft_tensor,inSideRight_tensor,OutSideRightAbove_tensor,OutSideLeftAbove_tensor), dim=1)

        # Passa as características concatenadas pela rede neural
        x = torch.relu(self.fc1(features))
        x = torch.sigmoid(self.fc2(x))
        return x



# Crie uma instância do predicado aprendido
pred_aprendido = PredicadoAprendido()

# Defina a função de perda
loss_fn = nn.MSELoss()

# Defina o otimizador
optimizer = optim.Adam(pred_aprendido.parameters(), lr=0.001)
# Crie uma instância do dataset
dataset = BoundingBoxDataset(num_samples=10)

# Loop de treinamento
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()

    # Ajuste do loop para iterar sobre o dataset
    for image, bbox in dataset:
        # Converte o formato dos dados para adequar ao modelo
        image = image.unsqueeze(0)  # Adiciona dimensão para batch
        bbox = bbox.unsqueeze(0)    # Adiciona dimensão para batch

        # Calcule a saída do predicado aprendido
        output = pred_aprendido(bbox, bbox)

        # Calcule a perda
        loss = loss_fn(output, bbox)  # Comparando com o próprio bbox para treinamento supervisionado

        # Realize a retropropagação e atualize os pesos
        loss.backward()
        optimizer.step()

    # Exiba a perda a cada epoch, se desejado
    if epoch % 100 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item()}')

print("Training finished at Epoch %d with Sat Level %.3f" %(epoch, 1 - loss.item()))

KeyboardInterrupt: 